In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import pandas as pd
%config InlineBackend.figure_format = 'svg' 

from clickhouse_driver import Client

client = Client('oleg.orbita.work', port = 9000, user='student_ml',
password='CGRV8zemLGgsfdz7Uc6ZJeisGTcPQa', settings={'use_numpy': True})
client.execute('SHOW DATABASES')

table_name = 'students_datas.weather_history'
df = client.query_dataframe(f"SELECT * FROM {table_name}")

In [ ]:
df.head()

2. удаление столбца idx


In [ ]:
df = df.drop('idx', axis=1)
df.head()

3.1 есть пропущенные значения в полях min_t, average_t, max_t, rainfall

3.2 больше всего в rainfall


In [ ]:
df.info()
df.isnull().sum()

4. больше всего пропусков в 1986 и 2003


In [ ]:
print(df.groupby('year').apply(lambda x: x.isna().sum().sum(),
      include_groups=False).sort_values(ascending=False))

предварительная обработка данных


In [ ]:
not_numeric = ['min_t', 'average_t', 'max_t', 'rainfall']
df.loc[:, not_numeric] = df.loc[:, not_numeric].replace(
    regex=True, to_replace=',', value='.')
df.loc[:, not_numeric] = df.loc[:, not_numeric].replace(
    regex=True, to_replace=r'^[-+]?(?:0-9+\.?0-9*|\.0-9+)$', value='')

for i in not_numeric:
    df[i] = pd.to_numeric(df[i], errors='coerce')
# df[df['min_t'].isna()]
df = df.dropna().drop_duplicates()

df.info(verbose=True)
df.describe().T

In [ ]:
df.isnull().sum()

5. новый столбец даты


In [ ]:
df['Date'] = pd.to_datetime(arg=df[['year', 'month', 'day']].astype(
    str).agg("-".join, axis=1), format="%Y-%m-%d")
df = df.sort_values('Date')
df.head()

6. размах температур и количество дней без осадков


In [ ]:
import itertools
df['Temp_range'] = df['max_t'] - df['min_t']


def get_days_without_rainfall(acc, x):
    return 0 if x > 0 else acc + 1

dfcopy2 = df.copy()
df['Without_rainfall'] = list(itertools.accumulate(
    df['rainfall'], get_days_without_rainfall, initial=0))[1:]
df

6 второй способ

In [ ]:
dfcopy2['Without_rainfall2'] = 0
counter = 0
for index, row in dfcopy2.iterrows():
    if row['rainfall'] > 0: counter = 0
    else: counter += 1
    dfcopy2.loc[index, 'Without_rainfall2'] = counter
dfcopy2 

7. самый длинный период засухи


In [ ]:
print(df['Without_rainfall'].max())

8. среднегодовая температура и сумма осадков по годам


In [ ]:
temp_8 = df.groupby('year')['average_t'].mean()
rainfall_8 = df.groupby('year')['rainfall'].sum()
plt.rcParams["figure.figsize"] = (13, 5)

In [ ]:
diag1 = temp_8.plot.bar()
diag1.legend(loc=4)

In [ ]:
diag2 = rainfall_8.plot.bar()
diag2.legend(loc=4)

In [ ]:
print(
    f'самый тёплый год - {temp_8.idxmax()}, самый холодный - {temp_8.idxmin()}')
print(
    f'год максимума осадков - {rainfall_8.idxmax()}, минимума - {rainfall_8.idxmin()}')

9.1 средняя температура ниже -30


In [ ]:
print(df[df['average_t'] < -30][['Date', 'average_t']])

9.2 средняя температура выше 27 и количество дней без осадков больше 3


In [ ]:
print(df[(df['average_t'] > 27) & (df['Without_rainfall'] > 3)]
      [['Date', 'average_t', 'Without_rainfall']])

In [ ]:
# with pd.option_context("display.min_rows", 50, "display.max_rows", 100, \
#    "display.max_columns", 15, 'display.max_colwidth', 150): display(df)